In [34]:
import pandas as pd
import Levenshtein as lv

In [4]:
import os

In [9]:
csd = os.getcwd()
print(csd)

/home/francesco/shared_files/python_notebooks/SD_for_HSS/Cycle-Data


In [8]:
os.chdir('/home/francesco/shared_files/python_notebooks/SD_for_HSS/Cycle-Data')

In [16]:
tt = pd.read_csv('./1-dataviz_tools/netherlands.csv')

In [17]:
len(tt), tt.head()

(1909,
   Flag of vessel Date vessel departed with captives          Vessel name  \
 0    Netherlands               1732-09-03T00:00:00Z            Waartwijk   
 1    Netherlands               1706-09-09T00:00:00Z        Wakende Kraan   
 2    Netherlands               1706-09-09T00:00:00Z        Wakende Kraan   
 3    Netherlands                                NaN  Wapen van Amsterdam   
 4    Netherlands                                NaN  Wapen van Amsterdam   
 
         Captain's name  
 0    Velde, Daniel ter  
 1  Hoeven, Jan van der  
 2       Wenman, Roelof  
 3                  NaN  
 4       Scheij, Pieter  )

In [60]:
tt.columns = ['flag','date_departed', 'name', 'captain']

tt.head()

,flag,date_departed,name,captain
0,Netherlands,1732-09-03T00:00:00Z,Waartwijk,"Velde, Daniel ter"
1,Netherlands,1706-09-09T00:00:00Z,Wakende Kraan,"Hoeven, Jan van der"
2,Netherlands,1706-09-09T00:00:00Z,Wakende Kraan,"Wenman, Roelof"
3,Netherlands,NaN,Wapen van Amsterdam,NaN
4,Netherlands,NaN,Wapen van Amsterdam,"Scheij, Pieter"


In [50]:
tts = ''.join(sorted(tt['name'][3].lower().split()))
print(tts)

amsterdamvanwapen


In [51]:
### https://www.statology.org/levenshtein-distance-in-python/
t1 = 'amsterdamwapen'
lv.distance(tts, t1)

3

In [64]:
tt=tt.fillna('')
tt.head()

,flag,date_departed,name,captain
0,Netherlands,1732-09-03T00:00:00Z,Waartwijk,"Velde, Daniel ter"
1,Netherlands,1706-09-09T00:00:00Z,Wakende Kraan,"Hoeven, Jan van der"
2,Netherlands,1706-09-09T00:00:00Z,Wakende Kraan,"Wenman, Roelof"
3,Netherlands,,Wapen van Amsterdam,
4,Netherlands,,Wapen van Amsterdam,"Scheij, Pieter"


In [65]:
tt.isna()

,flag,date_departed,name,captain
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
1904,False,False,False,False
1905,False,False,False,False
1906,False,False,False,False
1907,False,False,False,False


In [66]:
tt[tt["name"].str.contains('Wapen van Amsterdam')]

,flag,date_departed,name,captain
3,Netherlands,,Wapen van Amsterdam,
4,Netherlands,,Wapen van Amsterdam,"Scheij, Pieter"
937,Netherlands,1663-04-28T00:00:00Z,Wapen van Amsterdam,"Nuchteren, Jan Gerritsen"
938,Netherlands,1663-04-28T00:00:00Z,Wapen van Amsterdam,"Ridder, Paulus Heijn"
